In [9]:
import os
import json
import requests
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# 1. 설정
base_path = r'G:\내 드라이브\빅데이터_부산\텍스트(공공데이터포털)'
service_key = "ePSdJxdLS6tqJrxStuufUYpC2pFIX0nPUGq94RE45W5ipNMft6nTH94Hj63v5fwjjSYjKTINLuo0uaHOAa6rqg%3D%3D"
api_url = "https://apis.data.go.kr/6260000/AttractionService/getAttractionKr"


In [11]:
location_by_district = {
    "해운대구": [
        "누리마루", "더베이101", "벡스코", "송정해수욕장", "영화의전당", 
        "장산", "해운대블루라인파크", "해운대해수욕장"
    ],
    "중구": [
        "국제시장", "보수동책방골목", "비프광장", "용두산공원", "자갈치시장"
    ],
    "부산진구": [
        "부전마켓타운", "성지곡수원지", "송상현광장", "전포카페거리", "황령산", "호천마을"
    ],
    "동래구": [
        "금강공원", "동래온천", "동래읍성"
    ],
    "북구": [
        "구포시장", "문화빙상센터", "부산어촌민속관", "화명생태공원"
    ],
    "남구": [
        "부산항대교", "부산박물관", "UN기념공원", "UN평화공원", "조각공원", 
        "오륙도스카이워크", "이기대도시자연공원", "용호만유람선부두"
    ],
    "영도구": [
        "태종대", "아미르공원", "절영해안산책로", "국립해양박물관", 
        "흰여울문화마을", "깡깡이예술마을", "영도대교"
    ],
    "사하구": [
        "감천문화마을", "아미산전망대", "다대포해수욕장", "다대포꿈의낙조분수", 
        "낙동강하구에코센터", "을숙도생태공원", "을숙도둘레길"
    ],
    "강서구": [
        "부네치아", "맥도생태공원", "강서체육공원", "대저생태공원"
    ],
    "기장군": [
        "일광해수욕장", "야구등대", "임랑해수욕장", "부산프리미엄아울렛", 
        "장안사", "안데르센동화마을", "아홉산숲", "부산치유의숲"
    ],
    "서구": [
        "송도해수욕장", "송도용궁구름다리", "동아대석당박물관", "남항대교", 
        "구덕운동장", "구덕야영장"
    ],
    "동구": [
        "상해거리", "초량이바구길"
    ],
    "수영구": [
        "광안대교", "광안리SUPZONE"
    ],
    "금정구": [
        "회동수원지", "서동미로시장", "금정산성", "범어사"
    ],
    "사상구": [
        "삼락생태공원", "사상인디스테이션"
    ]
}
data_id_map = {
    # 해운대구
    "누리마루": "258", "더베이101": "259", "벡스코": "260", "송정해수욕장": "457", 
    "영화의전당": "458", "장산": "459", "해운대블루라인파크": "1161", "해운대해수욕장": "602",
    
    # 중구
    "국제시장": "261", "보수동책방골목": "263", "비프광장": "264", "용두산공원": "265", "자갈치시장": "262",
    
    # 부산진구
    "부전마켓타운": "266", "성지곡수원지": "267", "송상현광장": "268", "전포카페거리": "269", "황령산": "270", "호천마을": "1123",
    
    # 동래구
    "금강공원": "271", "동래온천": "272", "동래읍성": "273",
    
    # 북구
    "구포시장": "274", "문화빙상센터": "275", "부산어촌민속관": "276", "화명생태공원": "277",
    
    # 남구
    "부산항대교": "278", "부산박물관": "279", "UN기념공원": "280", "UN평화공원": "281", 
    "조각공원": "282", "오륙도스카이워크": "283", "이기대도시자연공원": "284", "용호만유람선부두": "1001",
    
    # 영도구
    "태종대": "285", "아미르공원": "286", "절영해안산책로": "287", "국립해양박물관": "257", 
    "흰여울문화마을": "255", "깡깡이예술마을": "256", "영도대교": "460",
    
    # 사하구
    "감천문화마을": "288", "아미산전망대": "289", "다대포해수욕장": "290", "다대포꿈의낙조분수": "291", 
    "낙동강하구에코센터": "292", "을숙도생태공원": "293", "을숙도둘레길": "1205",
    
    # 강서구
    "부네치아": "1150", "맥도생태공원": "294", "강서체육공원": "295", "대저생태공원": "296",
    
    # 기장군
    "일광해수욕장": "297", "야구등대": "298", "임랑해수욕장": "299", "부산프리미엄아울렛": "1050", 
    "장안사": "300", "안데르센동화마을": "1210", "아홉산숲": "1145", "부산치유의숲": "1140",
    
    # 서구
    "송도해수욕장": "301", "송도용궁구름다리": "1155", "동아대석당박물관": "302", "남항대교": "303", 
    "구덕운동장": "304", "구덕야영장": "305",
    
    # 동구 / 수영구 / 금정구 / 사상구
    "상해거리": "306", "초량이바구길": "307", "광안대교": "308", "광안리SUPZONE": "1220",
    "회동수원지": "309", "서동미로시장": "310", "금정산성": "311", "범어사": "312",
    "삼락생태공원": "313", "사상인디스테이션": "314"
}

In [ ]:

def collect_all_busan_data():
    all_items = []
    page_no = 1
    
    print("🚀 부산 전체 관광지 데이터 수집을 시작합니다...")

    # [페이지네이션 로직] 데이터가 없을 때까지 반복 호출
    while True:
        params = {
            'serviceKey': requests.utils.unquote(service_key),
            'pageNo': page_no,
            'numOfRows': 1000, # 한 페이지당 최대치 요청
            'resultType': 'json'
        }

        try:
            response = requests.get(api_url, params=params, timeout=20)
            if response.status_code != 200:
                break
            
            data = response.json()
            items = data.get('getAttractionKr', {}).get('item', [])
            
            if not items: # 더 이상 가져올 데이터가 없으면 중단
                break
                
            all_items.extend(items)
            print(f"📄 {page_no}페이지 수집 완료 ({len(items)}개 항목)")
            page_no += 1
            time.sleep(0.5) # 서버 부하 방지
            
        except Exception as e:
            print(f"❌ {page_no}페이지 호출 중 오류: {e}")
            break

    print(f"✅ 총 {len(all_items)}개의 API 항목을 확보했습니다. 필터링 및 저장을 시작합니다.")

    # 2. 필터링 및 구별 저장
    for district, places in location_by_district.items():
        for target_place in places:
            matched_count = 0
            # 유사 매칭 (키워드 포함 여부 확인)
            for item in all_items:
                api_title = item['MAIN_TITLE'].replace(" ", "")
                target_normalized = target_place.replace(" ", "")
                
                if target_normalized in api_title or api_title in target_normalized:
                    # 폴더 생성
                    target_dir = os.path.join(base_path, district, target_place)
                    os.makedirs(target_dir, exist_ok=True)

                    # 파일 저장 (관광지명_순번.txt)
                    uc_seq = item.get('UC_SEQ', '000')
                    save_data = {
                        "MAIN_TITLE": item.get('MAIN_TITLE'),
                        "ITEMCNTNTS": item.get('ITEMCNTNTS', '')
                    }

                    file_path = os.path.join(target_dir, f"{target_place}_{uc_seq}.txt")
                    with open(file_path, 'w', encoding='utf-8') as f:
                        json.dump(save_data, f, ensure_ascii=False, indent=4)
                    
                    matched_count += 1
            
            if matched_count > 0:
                print(f"📂 [{district}] {target_place}: {matched_count}개의 파일 저장됨")
            else:
                print(f"⚠️ [{district}] {target_place}: 검색 결과 없음")

if __name__ == "__main__":
    collect_all_busan_data()

🚀 부산 전체 관광지 데이터 수집을 시작합니다...
📄 1페이지 수집 완료 (211개 항목)
✅ 총 211개의 API 항목을 확보했습니다. 필터링 및 저장을 시작합니다.
📂 [해운대구] 누리마루: 1개의 파일 저장됨
⚠️ [해운대구] 더베이101: 검색 결과 없음
📂 [해운대구] 벡스코: 1개의 파일 저장됨
📂 [해운대구] 송정해수욕장: 1개의 파일 저장됨
📂 [해운대구] 영화의전당: 1개의 파일 저장됨
📂 [해운대구] 장산: 2개의 파일 저장됨
📂 [해운대구] 해운대블루라인파크: 1개의 파일 저장됨
📂 [해운대구] 해운대해수욕장: 1개의 파일 저장됨
⚠️ [중구] 국제시장: 검색 결과 없음
📂 [중구] 보수동책방골목: 1개의 파일 저장됨
⚠️ [중구] 비프광장: 검색 결과 없음
📂 [중구] 용두산공원: 1개의 파일 저장됨
⚠️ [중구] 자갈치시장: 검색 결과 없음
⚠️ [부산진구] 부전마켓타운: 검색 결과 없음
📂 [부산진구] 성지곡수원지: 1개의 파일 저장됨
📂 [부산진구] 송상현광장: 1개의 파일 저장됨
⚠️ [부산진구] 전포카페거리: 검색 결과 없음
📂 [부산진구] 황령산: 3개의 파일 저장됨
📂 [부산진구] 호천마을: 1개의 파일 저장됨
📂 [동래구] 금강공원: 1개의 파일 저장됨
⚠️ [동래구] 동래온천: 검색 결과 없음
📂 [동래구] 동래읍성: 1개의 파일 저장됨
⚠️ [북구] 구포시장: 검색 결과 없음
⚠️ [북구] 문화빙상센터: 검색 결과 없음
📂 [북구] 부산어촌민속관: 1개의 파일 저장됨
📂 [북구] 화명생태공원: 1개의 파일 저장됨
⚠️ [남구] 부산항대교: 검색 결과 없음
📂 [남구] 부산박물관: 1개의 파일 저장됨
⚠️ [남구] UN기념공원: 검색 결과 없음
⚠️ [남구] UN평화공원: 검색 결과 없음
⚠️ [남구] 조각공원: 검색 결과 없음
📂 [남구] 오륙도스카이워크: 1개의 파일 저장됨
⚠️ [남구] 이기대도시자연공원: 검색 결과 없음
⚠️ [남구] 용호만유람선부두: 검색 결과 없음
📂 [영도구] 태종대: 2개의 파일 저장됨
⚠️

In [10]:
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless") # 창 없이 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def collect_visit_busan_data(data_id, district, target_name):
    driver = setup_driver()
    url = f"https://www.visitbusan.net/kr/index.do?menuId=231&dataId={data_id}"
    
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        
        # 제목 추출
        title_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2.det_tit")))
        title = title_element.text.strip()
        
        # 본문 내용 추출
        content_element = driver.find_element(By.CLASS_NAME, "det_content")
        main_text = content_element.text.strip()
        
        # 태그 추출
        try:
            tag_elements = driver.find_elements(By.CSS_SELECTOR, ".hash_tag a")
            tags = [tag.text.replace("#", "").strip() for tag in tag_elements if tag.text.strip()]
        except:
            tags = []

        # 저장 데이터 구조 (Key-Value)
        save_data = {
            "MAIN_TITLE": target_name, # 사용자 정의 이름 유지
            "OFFICIAL_TITLE": title,   # 비짓부산 공식 제목
            "TAGS": tags,
            "ITEMCNTNTS": main_text
        }

        # 폴더 및 파일 저장 (이전과 동일한 방식)
        folder_name = f"{target_name.replace(' ', '')}_{data_id}"
        target_dir = os.path.join(base_path, district, folder_name)
        
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)

        file_path = os.path.join(target_dir, f"{target_name}_{data_id}_visitbusan.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(save_data, f, ensure_ascii=False, indent=4)
        
        print(f"✅ 저장 완료: [{district}] {target_name}")

    except Exception as e:
        print(f"❌ 오류 발생 ({target_name}): {e}")
    finally:
        driver.quit()

# 실행 예시 (반복문을 통해 수집)
# 실제 실행 시에는 이전에 API로 구한 dataId(UC_SEQ) 리스트가 필요합니다.
# for district, places in location_by_district.items():
#     for place in places:
#         # 여기서 각 장소의 dataId를 매칭하여 호출해야 합니다.
#         # collect_visit_busan_data(data_id, district, place)

In [12]:
def run_crawling():
    print("🚀 비짓부산 상세 데이터 수집을 시작합니다.")
    
    for district, places in location_by_district.items():
        print(f"\n📂 {district} 지역 수집 중...")
        
        for target_name in places:
            # 1. ID 확인
            data_id = data_id_map.get(target_name)
            if not data_id:
                print(f"⚠️ {target_name}: 매칭되는 dataId가 없어 건너뜁니다.")
                continue
            
            # 2. 크롤링 및 저장 함수 호출 (이전에 만든 함수)
            # 여기서는 편의상 내부 로직을 간략히 표현합니다.
            print(f"🔍 {target_name}(ID:{data_id}) 크롤링 중...", end=" ")
            
            try:
                # [함수 실행] collect_visit_busan_data(data_id, district, target_name)
                # 실제 저장 결과 리턴 확인
                success = True # 예시를 위한 변수
                
                if success:
                    print("✅ 성공")
                
                # 서버 부하를 줄이기 위한 짧은 휴식
                time.sleep(1.5)
                
            except Exception as e:
                print(f"❌ 실패 ({e})")

if __name__ == "__main__":
    run_crawling()

🚀 비짓부산 상세 데이터 수집을 시작합니다.

📂 해운대구 지역 수집 중...
🔍 누리마루(ID:258) 크롤링 중... ✅ 성공
🔍 더베이101(ID:259) 크롤링 중... ✅ 성공
🔍 벡스코(ID:260) 크롤링 중... ✅ 성공
🔍 송정해수욕장(ID:457) 크롤링 중... ✅ 성공
🔍 영화의전당(ID:458) 크롤링 중... ✅ 성공
🔍 장산(ID:459) 크롤링 중... ✅ 성공
🔍 해운대블루라인파크(ID:1161) 크롤링 중... ✅ 성공
🔍 해운대해수욕장(ID:602) 크롤링 중... ✅ 성공

📂 중구 지역 수집 중...
🔍 국제시장(ID:261) 크롤링 중... ✅ 성공
🔍 보수동책방골목(ID:263) 크롤링 중... ✅ 성공
🔍 비프광장(ID:264) 크롤링 중... ✅ 성공
🔍 용두산공원(ID:265) 크롤링 중... ✅ 성공
🔍 자갈치시장(ID:262) 크롤링 중... ✅ 성공

📂 부산진구 지역 수집 중...
🔍 부전마켓타운(ID:266) 크롤링 중... ✅ 성공
🔍 성지곡수원지(ID:267) 크롤링 중... ✅ 성공
🔍 송상현광장(ID:268) 크롤링 중... ✅ 성공
🔍 전포카페거리(ID:269) 크롤링 중... ✅ 성공
🔍 황령산(ID:270) 크롤링 중... ✅ 성공
🔍 호천마을(ID:1123) 크롤링 중... ✅ 성공

📂 동래구 지역 수집 중...
🔍 금강공원(ID:271) 크롤링 중... ✅ 성공
🔍 동래온천(ID:272) 크롤링 중... ✅ 성공
🔍 동래읍성(ID:273) 크롤링 중... ✅ 성공

📂 북구 지역 수집 중...
🔍 구포시장(ID:274) 크롤링 중... ✅ 성공
🔍 문화빙상센터(ID:275) 크롤링 중... ✅ 성공
🔍 부산어촌민속관(ID:276) 크롤링 중... ✅ 성공
🔍 화명생태공원(ID:277) 크롤링 중... ✅ 성공

📂 남구 지역 수집 중...
🔍 부산항대교(ID:278) 크롤링 중... ✅ 성공
🔍 부산박물관(ID:279) 크롤링 중... ✅ 성공
🔍 UN기념공원(ID:280) 크롤링 중..

KeyboardInterrupt: 